In [2]:
import requests

res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')

with open('nsmc_train.csv', 'wb') as f:
    f.write(res.content)

In [3]:
import pandas as pd 
nsmc = pd.read_csv('nsmc_train.csv', sep='\t')

In [5]:
# 0은 부정 1은 긍정
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# 보통 일반적으로 명사만 뽑아냄.. 

In [6]:
import stanza

In [7]:
nlp = stanza.Pipeline('ko')

2021-03-29 16:38:10 INFO: Loading these models for language: ko (Korean):
| Processor | Package |
-----------------------
| tokenize  | kaist   |
| pos       | kaist   |
| lemma     | kaist   |
| depparse  | kaist   |

2021-03-29 16:38:10 INFO: Use device: cpu
2021-03-29 16:38:10 INFO: Loading: tokenize
2021-03-29 16:38:10 INFO: Loading: pos
2021-03-29 16:38:10 INFO: Loading: lemma
2021-03-29 16:38:10 INFO: Loading: depparse
2021-03-29 16:38:11 INFO: Done loading processors!


In [37]:
text = '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'

In [38]:
doc = nlp(text)

In [39]:
# 문서에 있는 문장들을 리스트의 형태로 나눠 놓음 doc.sentences
doc.sentences

[[
   {
     "id": 1,
     "text": "교도소",
     "lemma": "교도소",
     "upos": "NOUN",
     "xpos": "ncn",
     "head": 2,
     "deprel": "compound",
     "misc": "start_char=0|end_char=3"
   },
   {
     "id": 2,
     "text": "이야기구먼",
     "lemma": "이야기구먼",
     "upos": "NOUN",
     "xpos": "ncn+ncn",
     "head": 3,
     "deprel": "compound",
     "misc": "start_char=4|end_char=9"
   },
   {
     "id": 3,
     "text": "..솔직히",
     "lemma": "..솔직히",
     "upos": "NOUN",
     "xpos": "nnc+nbu",
     "head": 4,
     "deprel": "compound",
     "misc": "start_char=10|end_char=15"
   },
   {
     "id": 4,
     "text": "재미는",
     "lemma": "재미+는",
     "upos": "NOUN",
     "xpos": "ncpa+jxt",
     "head": 5,
     "deprel": "dislocated",
     "misc": "start_char=16|end_char=19"
   },
   {
     "id": 5,
     "text": "없다",
     "lemma": "없+다",
     "upos": "ADJ",
     "xpos": "paa+ef",
     "head": 0,
     "deprel": "root",
     "misc": "start_char=20|end_char=22"
   },
   {
     "id": 6,
     "

In [40]:
for sentence in doc.sentences:
    for word in sentence.words:
        lemma = word.lemma.split('+')
        pos = word.xpos.split('+')
        for w, p in zip(lemma, pos):
            if p.startswith('n'):
                print(w)

교도소
이야기구먼
..솔직히
재미
.평점
조정


In [43]:
def extract_nouns(text):
    doc = nlp(text)
    for sentence in doc.sentences:
        for word in sentence.words:
            lemma = word.lemma.split('+')
            pos = word.xpos.split('+')
            for w, p in zip(lemma, pos):
                if p.startswith('n'):
                    yield w

In [44]:
list(extract_nouns(text))

['교도소', '이야기구먼', '..솔직히', '재미', '.평점', '조정']

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
cv = CountVectorizer(max_features=100, tokenizer=extract_nouns)

In [47]:
tdm = cv.fit_transform(nsmc.loc[0:100, 'document']) # loc 는 100 포함 

In [48]:
tdm.shape

(101, 100)

In [49]:
word_count = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '빈도': tdm.sum(axis=0).flat
})

In [51]:
word_count.sort_values('빈도', ascending=False)

,단어,빈도
47,영화,19
46,연기,6
49,완전,6
9,내용,5
92,진짜,5
...,...,...
61,은은하,1
62,은희,1
63,음내,1
26,신카이,1


In [ ]:
# stop word 목록을 만들어라?? ,,,, 흠... 

In [52]:
word_count.to_excel('nsmc_count.xlsx')